In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
import json
import sys
sys.path.insert(0, 'Data')
from config import g_key

#Store Part I results into DataFrame
#Load the data to a DataFrame
accident_data = pd.read_csv("Data/accident_data.csv", encoding="utf-8")
accident_data.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [30]:
# Data cleanup

# Filters out dates containing 2016, 2017, & 2019 -- too much data in dataset to handle with laptops
accident_data_revised = accident_data[~accident_data["Start_Time"].str.contains("2016")]
accident_data_revised = accident_data_revised[~accident_data_revised["Start_Time"].str.contains("2017")]
accident_data_revised = accident_data_revised[~accident_data_revised["Start_Time"].str.contains("2019")]

# Splits the Start_Time column into individual columns
accident_data_revised[["Date", "Time"]] = accident_data_revised["Start_Time"].str.split(expand=True)

# Renames columns to be more readable
accident_data_revised = accident_data_revised.rename(columns={"Start_Lat":"Lat", "Start_Lng":"Lng", "Weather_Condition":"Weather"})

# Filters and rearranges dataset to display most useful columns
accident_data_revised = accident_data_revised[["Date", "Time", "Lat", "Lng", "City", "State",
         "County", "Weather", "Temperature(F)", "Severity", "Nautical_Twilight"]]

# Iterates through all column data to find NaN values and adds them to a list
accident_data_revised.columns[accident_data_revised.isna().any()].tolist()

accident_data_revised = accident_data_revised.reset_index()

accident_data_revised.head(20)

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight
0,626043,2018-12-31,23:54:51,40.740047,-73.818512,Flushing,NY,Queens,Heavy Rain,46.9,3,Night
1,626045,2018-12-31,22:40:11,40.834370,-73.864113,Bronx,NY,Bronx,Rain,46.4,3,Night
2,626355,2018-12-31,12:35:54,38.441975,-88.953049,Dix,IL,Jefferson,Light Rain,57.0,3,Day
3,626407,2018-12-31,22:32:27,30.241417,-97.726158,Austin,TX,Travis,Partly Cloudy,39.9,2,Night
4,626462,2018-12-31,23:49:55,32.773060,-96.744247,Dallas,TX,Dallas,Clear,43.0,2,Night
5,626463,2018-12-31,23:49:24,32.818420,-96.802391,Dallas,TX,Dallas,Clear,43.0,2,Night
6,626563,2018-12-31,18:46:51,39.902153,-104.988914,Denver,CO,Adams,Snow,8.6,3,Night
7,626574,2018-12-31,22:26:47,32.192139,-110.857933,Tucson,AZ,Pima,Light Rain,37.4,2,Night
8,626599,2018-12-31,22:40:29,35.008537,-105.664352,Moriarty,NM,Torrance,Light Snow,27.9,3,Night
9,626602,2018-12-31,22:37:23,35.423836,-108.312744,Continental Divide,NM,McKinley,Light Snow,25.0,3,Night


In [31]:
# Combine Lat/Lng and City/State
accident_data_revised['Coordinates'] = accident_data_revised['Lat'].astype(str).str.zfill(2) + ', ' + accident_data_revised['Lng'].astype(str).str.zfill(3)

accident_data_revised['Location'] = accident_data_revised['City'] + ", " + accident_data_revised['State']
accident_data_revised.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight,Coordinates,Location
0,626043,2018-12-31,23:54:51,40.740047,-73.818512,Flushing,NY,Queens,Heavy Rain,46.9,3,Night,"40.740047, -73.818512","Flushing, NY"
1,626045,2018-12-31,22:40:11,40.834370,-73.864113,Bronx,NY,Bronx,Rain,46.4,3,Night,"40.83437, -73.864113","Bronx, NY"
2,626355,2018-12-31,12:35:54,38.441975,-88.953049,Dix,IL,Jefferson,Light Rain,57.0,3,Day,"38.441975, -88.953049","Dix, IL"
3,626407,2018-12-31,22:32:27,30.241417,-97.726158,Austin,TX,Travis,Partly Cloudy,39.9,2,Night,"30.241417, -97.726158","Austin, TX"
4,626462,2018-12-31,23:49:55,32.773060,-96.744247,Dallas,TX,Dallas,Clear,43.0,2,Night,"32.77306, -96.744247","Dallas, TX"


In [39]:
# Add column to dataframe showing frequency of accidents at specific coordinates
accident_data_revised['frequency'] = accident_data_revised['Coordinates'].map(accident_data_revised['Coordinates'].value_counts())
accident_data_revised.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight,Coordinates,Location,frequency
0,626043,2018-12-31,23:54:51,40.740047,-73.818512,Flushing,NY,Queens,Heavy Rain,46.9,3,Night,"40.740047, -73.818512","Flushing, NY",1
1,626045,2018-12-31,22:40:11,40.834370,-73.864113,Bronx,NY,Bronx,Rain,46.4,3,Night,"40.83437, -73.864113","Bronx, NY",3
2,626355,2018-12-31,12:35:54,38.441975,-88.953049,Dix,IL,Jefferson,Light Rain,57.0,3,Day,"38.441975, -88.953049","Dix, IL",1
3,626407,2018-12-31,22:32:27,30.241417,-97.726158,Austin,TX,Travis,Partly Cloudy,39.9,2,Night,"30.241417, -97.726158","Austin, TX",1
4,626462,2018-12-31,23:49:55,32.773060,-96.744247,Dallas,TX,Dallas,Clear,43.0,2,Night,"32.77306, -96.744247","Dallas, TX",1


In [40]:
# Which coordinates had the most severe accidents?
# Filter all rows for which severity is greater than or equal to 3
accident_data_revised_filtered = accident_data_revised[accident_data_revised['Severity'] >= 3] 
accident_data_revised_filtered.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight,Coordinates,Location,frequency
0,626043,2018-12-31,23:54:51,40.740047,-73.818512,Flushing,NY,Queens,Heavy Rain,46.9,3,Night,"40.740047, -73.818512","Flushing, NY",1
1,626045,2018-12-31,22:40:11,40.834370,-73.864113,Bronx,NY,Bronx,Rain,46.4,3,Night,"40.83437, -73.864113","Bronx, NY",3
2,626355,2018-12-31,12:35:54,38.441975,-88.953049,Dix,IL,Jefferson,Light Rain,57.0,3,Day,"38.441975, -88.953049","Dix, IL",1
6,626563,2018-12-31,18:46:51,39.902153,-104.988914,Denver,CO,Adams,Snow,8.6,3,Night,"39.90215300000001, -104.988914","Denver, CO",1
8,626599,2018-12-31,22:40:29,35.008537,-105.664352,Moriarty,NM,Torrance,Light Snow,27.9,3,Night,"35.008537, -105.664352","Moriarty, NM",1


In [42]:
# Which coordinates had the most frequent severe accidents?
# Filter all rows for which frequency is greater than 162
accident_data_most_severe = accident_data_revised_filtered[accident_data_revised_filtered['frequency'] >= 162]
accident_data_most_severe.head()

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight,Coordinates,Location,frequency
789,685846,2018-11-30,12:58:02,35.825504,-78.621483,Raleigh,NC,Wake,Mostly Cloudy,64.9,3,Day,"35.82550399999999, -78.621483","Raleigh, NC",172
815,685872,2018-11-30,15:24:31,35.814922,-78.605362,Raleigh,NC,Wake,Mostly Cloudy,64.9,3,Day,"35.814922, -78.605362","Raleigh, NC",189
891,685948,2018-11-30,22:05:03,35.825504,-78.621483,Raleigh,NC,Wake,Light Rain,57.0,3,Night,"35.82550399999999, -78.621483","Raleigh, NC",172
1061,686118,2018-11-30,07:58:57,34.833031,-82.296837,Greenville,SC,Greenville,Light Rain,50.0,3,Day,"34.833031, -82.296837","Greenville, SC",285
1062,686119,2018-11-30,07:59:16,34.858925,-82.259857,Greenville,SC,Greenville,Light Rain,50.0,3,Day,"34.858925, -82.259857","Greenville, SC",324


In [46]:
# Find unique coordinates per severity (this will delete some rows of data)
you_will_prob_die_here = accident_data_most_severe.drop_duplicates(subset=['Location'])
you_will_prob_die_here

,index,Date,Time,Lat,Lng,City,State,County,Weather,Temperature(F),Severity,Nautical_Twilight,Coordinates,Location,frequency
789,685846,2018-11-30,12:58:02,35.825504,-78.621483,Raleigh,NC,Wake,Mostly Cloudy,64.9,3,Day,"35.82550399999999, -78.621483","Raleigh, NC",172
1061,686118,2018-11-30,07:58:57,34.833031,-82.296837,Greenville,SC,Greenville,Light Rain,50.0,3,Day,"34.833031, -82.296837","Greenville, SC",285
1183,686240,2018-11-30,19:16:43,33.744976,-84.390343,Atlanta,GA,Fulton,Mostly Cloudy,64.9,3,Night,"33.744976, -84.390343","Atlanta, GA",206
2730,687787,2018-11-30,09:32:40,33.941364,-118.096634,Downey,CA,Los Angeles,Partly Cloudy,63.0,3,Day,"33.941364, -118.096634","Downey, CA",217
4108,689165,2018-12-02,18:05:17,30.422909,-91.139572,Baton Rouge,LA,East Baton Rouge,Clear,64.0,3,Night,"30.422909000000004, -91.139572","Baton Rouge, LA",188
4688,689745,2018-12-03,20:32:53,40.850067,-73.944817,New York,NY,New York,Partly Cloudy,48.0,3,Night,"40.850067, -73.944817","New York, NY",180
5517,690574,2018-12-03,17:30:32,33.978249,-81.195084,Lexington,SC,Lexington,Partly Cloudy,60.1,3,Day,"33.978249, -81.195084","Lexington, SC",184
5549,690606,2018-12-03,19:26:45,34.039364,-81.093620,Columbia,SC,Richland,Partly Cloudy,55.9,3,Night,"34.039364, -81.09362","Columbia, SC",205
6747,691804,2018-12-03,08:36:33,37.808498,-122.366852,San Francisco,CA,San Francisco,Mostly Cloudy,48.9,3,Day,"37.808498, -122.366852","San Francisco, CA",223
54789,739846,2018-10-31,17:20:09,44.966118,-93.269920,Minneapolis,MN,Hennepin,Partly Cloudy,53.1,3,Day,"44.966118, -93.26992","Minneapolis, MN",250


In [44]:
total_rows=len(you_will_prob_die_here.axes[0])
print(total_rows)

17
